In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
# Adam is faster than SGD (Stochastic Gradient Descent)
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

c:\python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
class LSTMbyHand(L.LightningModule):
    """
    Class created in order to initialize a LSTM RNN model that predicts a stock's value in the 5th day,
    having other 4 days as inputs
    """
    
    def __init__(self):
        """
        Create and initialize Weight and Bias tensors
        """
        # First, call the initialization method for the parent clas, LightningModule
        super().__init__()
        
        # We initialize the weights and biases around 0 (we make a normal distribution around 0 and a standard 
        # deviation around 1)
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)
        
        # We initialize the weights/biases for all the gates (all the cells in the LSTM unit)
        self.wlr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wlr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.blr1 = nn.Parameter(torch.tensor(0., requires_grad=True))
                                 
        self.wpr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wpr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bpr1 = nn.Parameter(torch.tensor(0., requires_grad=True))
                                 
        self.wp1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wp2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bp1 = nn.Parameter(torch.tensor(0., requires_grad=True))
                                
        self.wo1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wo2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bo1 = nn.Parameter(torch.tensor(0., requires_grad=True))
    
    def lstm_unit(self, input_value, long_memory, short_memory):
        """
        Do the LSTM math
        """
        # Calculates the percentage of the long-term memory to remember
        long_remember_percent = torch.sigmoid((short_memory * self.wlr1) + (input_value * self.wlr2) + self.blr1)
        
        # Creates a new, potential long-term memory and determines what percentage of it to remember
        potential_remember_percent = torch.sigmoid((short_memory * self.wpr1) + (input_value * self.wpr2) + self.bpr1)
        potential_memory = torch.tanh((short_memory * self.wp1) + (input_value * self.wp2) + self.bp1)
        
        # Update the long-term memory
        updated_long_memory = ((long_memory * long_remember_percent) + (potential_remember_percent * potential_memory))
        
        # Create a new short-term memory and determine what percentage to remember
        output_percent = torch.sigmoid((short_memory * self.wo1) + (input_value * self.wo2) + self.bo1)
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent
        
        # Return the long and short-term memories
        return([updated_long_memory, updated_short_memory])
    
    def forward(self, input):
        """
        Make a forward pass through unrolled LSTM
        """
        long_memory = 0
        short_memory = 0
        day1 = input[0]
        day2 = input[1]
        day3 = input[2]
        day4 = input[3]
        
        # Call LSTM with initial values (Day 1)
        long_memory, short_memory = self.lstm_unit(day1, long_memory, short_memory)
        
        # Call LSTM with values from Day 2
        long_memory, short_memory = self.lstm_unit(day2, long_memory, short_memory)
        
        # LSTM with values from Day 3
        long_memory, short_memory = self.lstm_unit(day3, long_memory, short_memory)
        
        # LSTM with values from Day 4
        long_memory, short_memory = self.lstm_unit(day4, long_memory, short_memory)
        
        return short_memory
    
    def configure_optimizers(self):
        """
        Configure Adam optimizer
        """
        return Adam(self.parameters())
    
    def training_step(self, batch, batch_idx):
        """
        Calculate loss and log training progress
        """
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i)**2
        
        self.log("train_loss", loss)
        
        if (label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)
            
        return loss

In [60]:
model = LSTMbyHand()
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.4568)


In [61]:
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Company B: Observed = 1, Predicted =  tensor(0.6164)


In [62]:
# Input data, days 1..4
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
# Label data, what we want to predict (0 for company A, 1 for company B)
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [50]:
# Training the model
trainer = L.Trainer(max_epochs=2000)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: C:\Users\Enachele\Desktop\Dissertatie\Program\Disertation_Program\PROGRAM_FINAL\indoor-localization-dnn-knn\pdr_estimation_lstm\lightning_logs

  | Name | Type | Params
------------------------------
------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total estimated model params size (MB)
c:\python39\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\python39\lib\site-packages\lightning\pytorch\loops\fit_loop.py:280: PossibleUserWarnin

Epoch 1999: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 83.35it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=2000` reached.


Epoch 1999: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 62.51it/s, v_num=0]


In [67]:
# After training for 2000 epochs
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.4568)


In [68]:
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Company B: Observed = 1, Predicted =  tensor(0.6164)


## In order to see how the epochs evolved, go to the folder with the ligning_logs file and input 
> vtensorboard --logdir=lightning_logs/

In [69]:
# Adding additional 1000 epochs to improve teh results
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path

trainer = L.Trainer(max_epochs=3000)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at C:\Users\Enachele\Desktop\Dissertatie\Program\Disertation_Program\PROGRAM_FINAL\indoor-localization-dnn-knn\pdr_estimation_lstm\lightning_logs\version_0\checkpoints\epoch=1999-step=4000.ckpt
c:\python39\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:337: UserWarning: The dirpath has changed from 'C:\\Users\\Enachele\\Desktop\\Dissertatie\\Program\\Disertation_Program\\PROGRAM_FINAL\\indoor-localization-dnn-knn\\pdr_estimation_lstm\\lightning_logs\\version_0\\checkpoints' to 'C:\\Users\\Enachele\\Desktop\\Dissertatie\\Program\\Disertation_Program\\PROGRAM_FINAL\\indoor-localization-dnn-knn\\pdr_estimation_lstm\\lightning_logs\\version_1\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `b

Epoch 2999: 100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 105.27it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=3000` reached.


Epoch 2999: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 79.98it/s, v_num=1]


In [70]:
# After training for 3000 epochs
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.0016)


In [71]:
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Company B: Observed = 1, Predicted =  tensor(0.9624)


## Now we create a new class that creates a LSTM model, this time with nn.lstm

In [73]:
class LightningLSTM(L.LightningModule):
    
    def __init__(self):
        """
        Create and initialize the LSTM unit
        """
        super().__init__()
        # input_size = number of features (only 1 for us)
        # hidden_size = the number of output values we want
        self.lstm = nn.LSTM(input_size=1, hidden_size=1)
        
    def forward(self, input):
        """
        Make a forward pass through unrolled LSTM
        """
        input_trans = input.view(len(input), 1)
        
        lstm_out, temp = self.lstm(input_trans)
        
        prediction = lstm_out[-1]
        return prediction
    
    def configure_optimizers(self):
        """
        Create optimizer with learning rate = 0.1
        """
        return Adam(self.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx):
        """
        Calculate loss and log training progress
        """
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i)**2
        
        self.log("train_loss", loss)
        
        if (label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)
            
        return loss

In [88]:
model = LightningLSTM()
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor([0.1081])


In [89]:
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Company B: Observed = 1, Predicted =  tensor([-0.0979])


In [90]:
# Lowered the number of epochs because we've increased the learning rate of the optimizer
trainer = L.Trainer(max_epochs=500, log_every_n_steps=2)

trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)


Epoch 499: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 95.23it/s, v_num=6]

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 71.42it/s, v_num=6]


In [91]:
model = LightningLSTM()
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor([0.0064])


In [92]:
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Company B: Observed = 1, Predicted =  tensor([0.0043])
